# Motivation

The developed approach requires the binned linkage-disequilibrium data (i.e. the expected $E[X_iX_jY_iY_j]$ for a pair of loci that are $u$ units apart). I provide two different tools to obtain such data: 

- A Python script `ldbucket_tskit.py` which takes as input a tree-sequence file (more convenient for simulated datasets)
- A compiled executable `ldbucket` which takes as input an indexed BCF or VCF.gz file (more convenient for real datasets)

Here, I simulate a minimal dataset and use both of them. 

In [1]:
params = {
    "sample_size" : 50,
    "Ne" : 2000,
    "L" : 1e8,
    'recombination_rate' : 1e-8,
    'mutation_rate' : 1e-8,
    'seed' : 1234
}

## Geneology simulation

In [2]:
import msprime

ts = msprime.sim_ancestry(
    samples=params['sample_size'],
    population_size=params['Ne'],
    sequence_length=params['L'],
    recombination_rate=params['recombination_rate'],
    random_seed=params['seed']
)
mts = msprime.sim_mutations(
        ts,
        rate=params['mutation_rate'],
        random_seed=params['seed']
    )
mts

## From `tskit` interactively

In [3]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))
import ldbucket_tskit

In [4]:
?ldbucket_tskit.ldbucket

Signature:
ldbucket_tskit.ldbucket(
    ts: tskit.trees.TreeSequence,
    recombination_rate=1e-08,
    maf_threshold=0.25,
    name='tskit',
) -> pandas.core.frame.DataFrame
Docstring:
Function for processing linkage disequilibrium data from `tskit` data. Convenient when working with simulated datasets.

Arguments:
    - ts: tskit.TreeSequence: Tree sequence object containing the genetic data.
    - recombination_rate: float: Recombination rate per base pair per generation.
    - maf_threshold: float: Minor allele frequency threshold for filtering sites.
    - name: str: Name of the contig (provided by the user).

- Returns a pandas DataFrame with the following columns:
    - 'contig_name': name of the contig (provided by the user)
    - 'bin_index': index of the bin (starting from 0)
    - 'left_edge': left edge of the bin in Morgan
    - 'right_edge': right edge of the bin in Morgan
    - 'mean_ld': mean linkage disequilibrium within the bin (computed as E[X_iX_jY_iY_j])
File:      

In [5]:
df_tskit = ldbucket_tskit.ldbucket(mts, recombination_rate=params['recombination_rate'])
df_tskit.head()

,contig_name,bin_index,left_bin,right_bin,mean
0,tskit,0,0.005,0.010,0.019701
1,tskit,1,0.010,0.015,0.011965
2,tskit,2,0.015,0.020,0.008895
3,tskit,3,0.020,0.025,0.007160
4,tskit,4,0.025,0.030,0.005816


## From `tskit` via the command line

In [6]:
mts.dump("example.trees")

In [7]:
%%bash
time python ../src/ldbucket_tskit.py example.trees tskit 1e-8 0.25 > example_tskit.csv


real	0m4.099s
user	0m3.964s
sys	0m0.096s


## From `BCF/VCF` via the command line

In [8]:
%%bash
tskit vcf example.trees | bcftools view -O b > example.bcf
bcftools index example.bcf

In [9]:
%%bash
./ldbucket --help

Computes observed linkage disequilibrium between all pairs of variants

Usage: ldbucket [OPTIONS] <INFILE>

Arguments:
  <INFILE>  Compressed and indexed VCF or BCF (better!) file

Options:
      --cores <CORES>
          Number of cores (only used if more than one contig is processed) [default: 1]
      --recombination-rate <RECOMBINATION_RATE>
          Recombination rate per base pair [default: 0.00000001]
      --maf-threshold <MAF_THRESHOLD>
          Minor allele frequency threshold [default: 0.25]
      --contig-names <CONTIG_NAMES>
          List of contig names (optional). If not provided, all contigs will be processed
      --sample-names <SAMPLE_NAMES>
          List of sample names (optional). If not provided, all samples will be processed
      --use-precomputed-maf
          Whether to use a precomputed MAF TAG. Be careful, this can lead to incorrect results if the MAF TAG is not accurate
  -h, --help
          Print help
  -V, --version
          Print version

The outpu

In [10]:
%%bash
time ./ldbucket example.bcf --recombination-rate 1e-8 > example_bcf.csv


real	0m1.622s
user	0m1.601s
sys	0m0.019s


## Comparison

As a sanity check, we next test whether both methods produce the same result (up to floating-point errors). 

In [11]:
import pandas as pd
df_bcf = pd.read_csv("example_bcf.csv")
df_bcf.head()

,contig_name,bin_index,left_bin,right_bin,mean,var,N
0,1,0,0.005,0.010,0.019701,0.003025,431346
1,1,1,0.010,0.015,0.011967,0.001958,424454
2,1,2,0.015,0.020,0.008893,0.001608,430962
3,1,3,0.020,0.025,0.007160,0.001450,426794
4,1,4,0.025,0.030,0.005816,0.001278,414422


In [15]:
import numpy as np
diff = np.abs(df_bcf['mean'] - df_tskit['mean'])
diff = diff[np.isfinite(diff)] # There might be bins without SNPs
assert np.all(diff<1e-5)